In [97]:
#1-2. Web URL input and Feature Selection
print("Running web url input and Feature Selection")

import pandas as pd
import requests
from bs4 import BeautifulSoup
import whois
from datetime import datetime
import socket
import ssl
from urllib.parse import urlparse
import requests
from urllib3.exceptions import NewConnectionError, MaxRetryError
from requests.exceptions import ConnectionError
import time
import re

# Define headers to mimic a real browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Referer': 'https://google.com',  # Optional

}

# Fetch URL with retries
def fetch_url(url, retries=2):
    for attempt in range(retries):
        try:
            response = requests.get(url, timeout=10)
            response.raise_for_status()  # Raise an HTTPError for bad responses (4xx, 5xx)
            return response.content
        except (requests.exceptions.RequestException, NewConnectionError, MaxRetryError, ConnectionError):
            # Suppress error details and retry
            time.sleep(2)  # Delay between retries
    # If all retries fail, return None
    return None

                
#------------------------------------------------------------------------------------------------------------------
                                             #Domain-based Feature
# Function to check SSL certificate
def check_ssl(domain):
    try:
        context = ssl.create_default_context()
        with socket.create_connection((domain, 443)) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as secure_sock:
                return True
    except:
        return False
#------------------------------------------------------------------------------------------------------------------
                                           

                                           
#-------------------------------------------------------------------------------------------------------------------
                                               #URL-based Feature
# Check for IP address in URL
def contains_ip(url):
    ip_pattern = re.compile(r'\b(?:[0-9]{1,3}\.){3}[0-9]{1,3}\b')
    return bool(ip_pattern.search(url))

#--------------------------------------------------------------------------------------------------------------------
                                              #URL-based feature
# Compile the shortening services regex pattern
shortening_services_pattern = re.compile(r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|"
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|"
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|"
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|"
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|"
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|"
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|"
                      r"tr\.im|link\.zip\.net")

# Count special characters in URL
def count_special_chars(url):
    return int(sum(not c.isalnum() and c not in ['.', '-', '_', ':', '/', '?', '&', '=', '%'] for c in url))

# Check for URL shortening services using the provided regex pattern
def shortening_services(url):
    return bool(re.search(shortening_services_pattern, url))

#----------------------------------------------------------------------------------------------------------------------

# Extract features from URL
def extract_features(url):
    features = {}
    domain = urlparse(url).netloc
    #-----------------------------------------------------------------------------------------------------------------
                                              #URL-based features
    features['url_length'] = int(len(url))
    features['contains_ip'] = int(contains_ip(url))
    features['shortening_services'] = int(shortening_services(url))
    features['special_chars'] = int(count_special_chars(url))
    #-----------------------------------------------------------------------------------------------------------------
    content = fetch_url(url)
    if content:
        soup = BeautifulSoup(content, 'html.parser')
        #-------------------------------------------------------------------------------------------------------------
                                              #Content-based features
        features['html_length'] = int(len(content))
        features['js_length'] = sum(len(s.string) for s in soup.find_all('script') if s.string)
        features['num_links'] = len(soup.find_all('a'))
        features['num_forms'] = len(soup.find_all('form'))
        
       
        #-------------------------------------------------------------------------------------------------------------
    
        try:
            #---------------------------------------------------------------------------------------------------------
                                              #Domain-based features
            domain_info = whois.whois(domain)
            creation_date = domain_info.creation_date
            updated_date = domain_info.updated_date
            expiration_date = domain_info.expiration_date
            
            if isinstance(creation_date, list):
                creation_date = creation_date[0]
            if isinstance(updated_date, list):
                updated_date = updated_date[0]
            if isinstance(expiration_date, list):
                expiration_date = expiration_date[0]
            
            features['domain_age'] = (datetime.now() - creation_date).days if creation_date else None
        except Exception as e:
            print(f"Error fetching domain info for {domain}: {e}")
            features['domain_age'] = None

        features['has_ssl'] = 1 if int(check_ssl(domain)) else 0
        #------------------------------------------------------------------------------------------------------------------

        features.update(features)
    else:
        features.update({
            'html_length': 0,
            'js_length': 0,
            'num_links': 0,
            'num_forms': 0,
            'contains_ip': 0,
            'shortening_services': 0,
            'url_length': len(url),
            'special_chars': sum(not c.isalnum() and c not in ['.', '-', '_'] for c in url),
            'domain_age': None,
            'has_ssl': 0,
        })
    return features

Running web url input and Feature Selection


In [99]:
                                              #3. Feature Vector
print("Extract features running")
# Function to process a single URL
def process_url(url):
    try:
        features = extract_features(url)
        return {'url': url, **features}
    except Exception as e:
        print(f"Error processing URL: {url}. Error: {e}")
        return None


Extract features running


In [101]:
                             #5. Data Preprocessing - Randomly process 200 URLs for each label
import pandas as pd
import random
from concurrent.futures import ThreadPoolExecutor, as_completed

print("Data preprocessing running")
def preprocess_data(df, processed_df, n=1000):
    # Separate URLs by label (0 for legitimate, 1 for phishing)
    legitimate_urls = df[df['status'] == 0]['url'].tolist()
    phishing_urls = df[df['status'] == 1]['url'].tolist()
    
    # Filter out URLs that have already been processed
    already_processed_urls = set(processed_df['url'].tolist()) if not processed_df.empty else set()
    legitimate_urls = [url for url in legitimate_urls if url not in already_processed_urls]
    phishing_urls = [url for url in phishing_urls if url not in already_processed_urls]

    # Randomly sample 1000 URLs from each group (or fewer if not enough remain)
    legitimate_sample = random.sample(legitimate_urls, min(n, len(legitimate_urls)))
    phishing_sample = random.sample(phishing_urls, min(n, len(phishing_urls)))

    all_samples = legitimate_sample + phishing_sample
    results = []
    
    # Multithreading to process URLs faster
    with ThreadPoolExecutor(max_workers=50) as executor:
        futures = {executor.submit(process_url, url): url for url in all_samples}
        for future in as_completed(futures):
            url = futures[future]
            try:
                result = future.result()
                if result:  # Only append if the URL is successfully processed
                    results.append(result)
                    print(f"Processed URL: {url}")
            except Exception as e:
                print(f"Error processing URL: {url}. Error: {e}")

    # Convert results to DataFrame
    X = pd.DataFrame(results)
    
    # Ensure the corresponding labels match the processed URLs
    y = df.loc[df['url'].isin(X['url']), 'status'].values

    return X, y

Data preprocessing running


In [103]:
print("Loading Historical Dataset")
import pandas as pd
import numpy as np

# Load datasets
df = pd.read_csv('legitimate_and_phishing_urls.csv')

# Replace "legitimate" with 0 and "phishing" with 1
# Use np.where() to replace "legitimate" with 0 and "phishing" with 1
df['status'] = np.where(df['status'] == 'legitimate', 0, 1)

#remove duplicates
df = df.drop_duplicates(subset='url')  # Remove duplicates

# Assuming we have a dataframe 'processed_df' of already processed URLs, or create an empty one
processed_df = pd.DataFrame(columns=['url'])

# Preprocess the data (sample 1000 legitimate and 200 phishing URLs for feature extraction)
X, y = preprocess_data(df, processed_df, n=1000)

# At this point, `X` contains the feature set and `y` contains the corresponding labels for training
print(f"Feature matrix shape: {X.shape}")
print(f"Labels shape: {len(y)}")
# X and y can now be used for model training

Loading Historical Dataset


2024-09-14 08:46:32,709 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno -2] Name or service not known


Processed URL: http://www.bastelitis.de/
Processed URL: http://jordanpost.com.jo
Processed URL: http://everything.explained.today/%5C/cladism/
Processed URL: http://www.battingcagesofmn.com/batting-cages/
Processed URL: http://musinfo.ch
Processed URL: https://en.wikipedia.org/wiki/MultiMediaCard
Processed URL: https://www.torproject.org/download/download
Error fetching domain info for www.cloudaccess.net: No match for "WWW.CLOUDACCESS.NET".
>>> Last update of whois database: 2024-09-14T07:46:14Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsoring
registrar.  Users may consult the sponsoring registrar's Whois database to
view the registrar's reported date of expiration for this registration.

TERMS OF USE: You are not authorized to access or query

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Processed URL: https://www.gamereactor.se/
Processed URL: https://en.m.wikipedia.org/wiki/House_Intelligence_Committee
Processed URL: http://www.yourdictionary.com/provenance
Processed URL: https://www.facebook.com/TheGraphicsFairy/
Processed URL: http://www.wellfloured.com/
Processed URL: https://www.facebook.com/DiscFamily/
Processed URL: https://www.taphunter.com/
Processed URL: https://www.cloudaccess.net/cloud-control-panel-ccp/157-dns-management/319-resolving-dns-http-redirect-issues.html
Processed URL: http://www.newworldencyclopedia.org/entry/CD
Processed URL: http://stackoverflow.com/questions/6260756/how-to-stop-javascript-foreach
Processed URL: https://en.wikipedia.org/wiki/List_of_CNN_personnel
Processed URL: https://www.youtube.com/channel/UCLSjw7g5U48jgtjaeyjfwjQ
Processed URL: https://www.newlifepca.org/
Processed URL: https://www.ovcmt.com/
Processed URL: http://zlsa.github.io/atc/
Processed URL: http://www.iconmeals.com/product-category/snacks/
Processed URL: https://w

...

Processed URL: http://harcomputer.com/http/match
Processed URL: https://coliissimo-livraison.com/colissimo%20new/colissimo/account/loading.html?id=300000716523000091
Processed URL: http://atpscan.global.hornetsecurity.com/index.php?atp_str=iwbJSC5r_fvjexWtZ0APVWAaNFBb5w5RaWbWfiVG2tJ7l3sMCHkJIzo6IzYwZmRmNjNlOWYwMSM6OiOB3TRcMGuLLky4VNdgcmDW
Processed URL: http://attys.yolasite.com/
Processed URL: https://financiallifecoaching.builderallwp.com/wp-content/themes/skanda/?qzwoiniuGYTRYwysvwedbygVDRwydgCCYyueuuyywUBYVYUbbxuWu
Processed URL: http://qubectravel.com/bpm44/
Processed URL: http://igsign00.000webhostapp.com/login.html
Processed URL: http://fbproject4df3409fkl342ef043.el.r.appspot.com/wp-content
Processed URL: http://www.goodhopeservices.com/update/
Processed URL: https://support-appleld.com.secureupdate.duilawyeryork.com/ap/e909aa5608f4fea/?cmd=_update&dispatch=e909aa5608f4fea21&locale=_US
Processed URL: http://mne.edu.vn/wp-includes/Netflix/LoginID/index.php
Processed URL: http://

2024-09-14 08:53:17,964 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno -2] Name or service not known


Processed URL: https://eoexpo.sa/boa/bankofamerica/next3.php
Error fetching domain info for interbahiss1.blogspot.com: No match for "INTERBAHISS1.BLOGSPOT.COM".
>>> Last update of whois database: 2024-09-14T07:53:05Z <<<

NOTICE: The expiration date displayed in this record is the date the
registrar's sponsorship of the domain name registration in the registry is
currently set to expire. This date does not necessarily reflect the expiration
date of the domain name registrant's agreement with the sponsoring
registrar.  Users may consult the sponsoring registrar's Whois database to
view the registrar's reported date of expiration for this registration.

TERMS OF USE: You are not authorized to access or query our Whois
database through the use of electronic processes that are high-volume and
automated except as reasonably necessary to register domain names or
modify existing registrations; the Data in VeriSign Global Registry
Services' ("VeriSign") Whois database is provided by VeriSign f

2024-09-14 08:54:46,990 - whois.whois - ERROR - Error trying to connect to socket: closing socket - [Errno -2] Name or service not known


Processed URL: http://www.peoplemakingplaces.com/includes/Support/En/log/signin/customer_center/customer-IDPP00C644/myaccount/signin/
Processed URL: http://www.stolizaparketa.ru/wp-content/themes/twentyfifteen/css/read/chinavali/index.php?email=abuse@fit-online.com
Processed URL: http://www.shadetreetechnology.com/V4/validation/ef27f396982a200c0e512f941f4653c2
Processed URL: http://y9o5m.codesandbox.io/
Processed URL: http://dscsc.lk/images/comunicado-pbb.com.br/
Processed URL: http://apple.com.icloud-lost.co/
Processed URL: http://piponi.com/58CEBC670A93888DE49B5C0EE755326F?sec=Michaela%20Fellenberg
Processed URL: http://marcoislandfurnitureanddesign.com/wp-includes/Text/Diff/Renderer/index.php
Processed URL: http://starmak.com.tr/950CAAEA0281AA2BEBED8F9ECCA4BD30/?sec=KnelsPatrick
Processed URL: https://worldativelion.my-free.website/
Processed URL: https://wh494528.ispot.cc/index.php?email=daesankh@hanmail.net
Processed URL: http://labanquepostale-ssm-voscomptepostalessl.com/
Process

In [104]:
                                       #5. Data Preprocessing
print("Running Data Preprocessing")

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

# Preprocess features
processed_df = pd.DataFrame(X)
processed_df['status']= y
processed_df.head()

Running Data Preprocessing


,url,url_length,contains_ip,shortening_services,special_chars,html_length,js_length,num_links,num_forms,domain_age,has_ssl,status
0,http://www.bastelitis.de/,25,0,0,0,63203,3155,28,0,NaN,0,0
1,http://jordanpost.com.jo,24,0,1,0,1686,405,0,1,NaN,1,0
2,http://everything.explained.today/%5C/cladism/,46,0,0,7,0,0,0,0,NaN,0,0
3,http://www.battingcagesofmn.com/batting-cages/,46,0,0,5,0,0,0,0,NaN,0,1
4,http://musinfo.ch,17,0,0,3,0,0,0,0,NaN,0,1


In [178]:
                                        #5. Data Preprocessing
# Remove rows with any NaN values in the dataframe
processed_df = processed_df.dropna()

# Check if there are any NaN values left (should print 0 for all columns)
print(processed_df.isnull().sum())

# Verify if any NaN values remain
print(f"Remaining NaN values: {processed_df.isna().sum().sum()}")  # This should print 0 if no NaNs are left
#save to csv file

processed_df.head()

url            0
url_length     0
html_length    0
js_length      0
num_links      0
num_forms      0
domain_age     0
has_ssl        0
status         0
dtype: int64
Remaining NaN values: 0


,url,url_length,html_length,js_length,num_links,num_forms,domain_age,has_ssl,status
0,https://www.torproject.org/download/download,44,29592,876,109,0,6541.0,1,0
1,https://www.gamereactor.se/,27,520543,99233,449,1,7623.0,1,0
2,https://en.m.wikipedia.org/wiki/House_Intellig...,60,128572,23253,292,1,8645.0,1,0
3,http://www.yourdictionary.com/provenance,40,276433,16364,130,1,9098.0,1,0
4,http://www.wellfloured.com/,27,163,0,0,0,3414.0,0,0


In [121]:
#Drop specified columns from the DataFrame
processed_df = processed_df.drop(columns=['contains_ip', 'shortening_services', 'special_chars'])
processed_df.head()

,url,url_length,html_length,js_length,num_links,num_forms,domain_age,has_ssl,status
0,https://www.torproject.org/download/download,44,29592,876,109,0,6541.0,1,0
1,https://www.gamereactor.se/,27,520543,99233,449,1,7623.0,1,0
2,https://en.m.wikipedia.org/wiki/House_Intellig...,60,128572,23253,292,1,8645.0,1,0
3,http://www.yourdictionary.com/provenance,40,276433,16364,130,1,9098.0,1,0
4,http://www.wellfloured.com/,27,163,0,0,0,3414.0,0,0


In [176]:
                                              #5. Data Preprocessing
#Seperate the legitimate represented as 0 and phishing representd as 1 dataframes
legit_df = processed_df[processed_df['status'] == 0].reset_index(drop=True)  # Rows with status == 0
phish_df = processed_df[processed_df['status'] == 1].reset_index(drop=True)    # Rows with status == 1
#concat the legitimate and phishing
# Concatenate legitimate_df and phishing_df
processed_df = pd.concat([legit_df, phish_df], ignore_index=True)


In [207]:
                              #6-7. Feature Scaling and Feature Selection

from imblearn.over_sampling import SMOTE

try:
    # Split data
    X = processed_df.drop(columns=['url', 'status'])
    y = processed_df['status']
except: 
    print('url and staus have been dropped and not found in the dataframe')
finally:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [127]:
                #8. Machine Learning Models such as RandomForest, Support Vector, and GradentBoosting

#Define classifiers
classifiers = {
    'RandomForest': RandomForestClassifier(),
    'SVM': SVC(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [129]:
                                            #9. Hyperparameter Tuning
param_grids = {
    'RandomForest': {'n_estimators': [100, 200], 'max_depth': [10, 20]},
    'SVM': {'C': [0.1, 1, 10], 'kernel': ['rbf', 'linear']},
    'GradientBoosting': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]}
}

In [131]:
                              #10. Model selection, training and evaluation metrics
best_model = None
best_score = 0

# Model selection, training and evaluation
for name, clf in classifiers.items():
    print(f"Training {name}...")
    
    grid_search = GridSearchCV(clf, param_grids[name], cv=5, scoring='accuracy')
    grid_search.fit(X_train_scaled, y_train)
    
    model = grid_search.best_estimator_
    y_pred = model.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    print(f"{name} - Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}, F1 Score: {f1}")
    
    if accuracy > best_score:
        best_score = accuracy
        best_model = model

Training RandomForest...
RandomForest - Accuracy: 0.563953488372093, Precision: 0.5802469135802469, Recall: 0.5340909090909091, F1 Score: 0.5562130177514792
Training SVM...
SVM - Accuracy: 0.4476744186046512, Precision: 0.40540540540540543, Recall: 0.17045454545454544, F1 Score: 0.24
Training GradientBoosting...
GradientBoosting - Accuracy: 0.5116279069767442, Precision: 0.5217391304347826, Recall: 0.5454545454545454, F1 Score: 0.5333333333333333


In [133]:
                                       #11. Final Model Selection
print(f"Best model: {best_model}")

Best model: RandomForestClassifier(max_depth=10)


In [140]:
                                       #12. Classification
def classify_url(url, model, scaler, feature_columns_order):
    features = extract_features(url)
    features_df = pd.DataFrame([features])
    # Ensure all required columns are present
    for col in feature_columns_order:
        if col not in features_df.columns:
            features_df[col] = 0
    
    features_df = features_df[feature_columns_order]
    
    # Remove any NaN values
    
    # Scale and predict
    features_scaled = scaler.transform(features_df)
    prediction = model.predict(features_scaled)
    print(prediction)
    return "Legitimate" if prediction == 0 else "Phishing"

In [191]:
# Enter new url
new_url = input("https://: ")


https://:  https://gmail.com


In [193]:
classification_result = classify_url(new_url, best_model, scaler, X.columns)
print(f"The URL '{new_url}' is classified as: {classification_result}")

[0]
The URL 'https://gmail.com' is classified as: Legitimate


In [209]:
#Import the model to use to create a web interface
import pickle

# Save the model
with open('model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

In [211]:
# Save the scaler
with open('scaler.pkl', 'wb') as scaler_file:
    pickle.dump(scaler, scaler_file)

In [215]:
X.to_csv('processed_data.csv', index=False)